# Видобуток та збагачення структурованих даних

### ТЕОРЕТИЧНА ЧАСТИНА ТА ПРИКЛАДИ

Для роботи з структурованими табличними даними, які розміщені в реляційній БД [SQLite](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiUyMOv1orvAhWDK3cKHac9DyMQFjAAegQIBxAD&url=https%3A%2F%2Fru.wikipedia.org%2Fwiki%2FSQLite&usg=AOvVaw1s-RMK2VC2tXyDZxZXDYDo) в Python передбачена стандартна бібліотека [sqlite3](https://docs.python.org/3/library/sqlite3.html)

In [ ]:
# підключення бібліотеки
import sqlite3

#### ПРИКЛАД ВИЛУЧЕННЯ ДАНИХ З БД
Створити БД SQLite з ім'ям `FIT-4m`, та імпорувати в неї таблицю, що містить журнал групи `jornal`

In [ ]:
# створюємо з'єднання з БД
conn = sqlite3.connect("FIT-4m.db")

[імпорт csv файла в sqlite через CLI](https://www.sqlitetutorial.net/sqlite-import-csv/) 

In [ ]:
# створюємо об'єкт `cursor`, що відповідає за реалізацію операцій з таблицями БД
cur = conn.cursor()

In [ ]:
print(type(cur), end='\n\n')
print(dir(cur))

##### виведемо ВСІ записи таблиці `jornal`

In [ ]:
# створимо інстанс для операції `SELECT`
students_cur = cur.execute("SELECT * FROM jornal")

# будуємо список записів за допомогою метода `fetchall()`
students_list = students_cur.fetchall()

_ = [print(x) for x in students_list]

#### ПРИКЛАД ЗБАГАЧЕННЯ ДАНИХ З ЗОВНІШНЬОГО ДЖЕРЕЛА

Додати до списка студентів їх оцінки за вступне тестування, які розташовані в csv файлі `test_result.csv`

імпортуємо `test_result.csv` в робочу БД як таблицю `test`

[імпорт csv файла в sqlite через CLI](https://www.sqlitetutorial.net/sqlite-import-csv/) 

##### Cтворимо нову таблицю, яка буде містити вміст таблиці `jornal` та поле оцінок з таблиці `test`



In [ ]:
# створимо вираз на побудову пустої таблиці `jornal_test` з відповідними полями
sql_stmt = "CREATE TABLE IF NOT EXISTS jornal_test \
              (Surname TEXT,         \
               Name TEXT,            \
               Email TEXT,           \
               GitHub TEXT,          \
               Test_result INT);"

# виконаемо запит на створення таблиці
cur.execute(sql_stmt)

In [ ]:
# перевіримо стан виконання операції - виведемо список таблиць
tables_list = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables_list.fetchall()

In [ ]:
# створимо запит на об'єднання таблиць 'jornal' та 'test' по прізвищу студента
sql_stmt = "SELECT jornal.*, test_result.test FROM  \
            jornal LEFT JOIN test_result ON jornal.Surname = test_result.Surname "

In [ ]:
# для перевірки виконаємо і виведемо результати запиту
cur.execute(sql_stmt).fetchmany(7)

In [ ]:
# заповнимо пусту таблицю результатми  запиту на злиття
_ = cur.execute("INSERT INTO jornal_test " + sql_stmt)

In [ ]:
# збережемо вміст нової таблиці в список
jornal_test_list = cur.execute("SELECT * FROM jornal_test").fetchall()

_ = [print(x) for x in jornal_test_list]

In [ ]:
# завершимо всі транзакції
conn.commit()

# закріємо з'єднання
conn.close()

#### ПРИКЛАД  ДЕСКРИПТИВНОГО АНАЛІЗУ ДАНИХ

Провести [попередній аналіз](https://ru.wikipedia.org/wiki/Описательная_статистика) отриманих даних з ціллю виявленя відхилень, помилок та інших непридатних даних.

Сведемо результати аналізу в таблицю, яка має наступний вигляд:

№| Показчик | Значення
:--:|:-------|-------:
1| кількість спостережень | xx 
2| кількість пустих значень | xx
4| середній бал|  xx.x
5| максимальний бал | xx
6| мінімальний бал  | xx
7| стандартне відхилення | xx.x
8| розмах вариації  | xx


In [ ]:
# підключимо бібліотеку 'numpy' і дамо їй аліас 'np'
import numpy as np

ознайомитись з призначенням, можливостями та основним функціями [бібліотеки numpy](https://numpy.org) 

In [ ]:
# перетворимо список студетів в numpay матрицю
jornal_test_arr = np.array(jornal_test_list)

In [ ]:
print(type (jornal_test_arr), jornal_test_arr.shape)

In [ ]:
# створимо масив с результатами тестів 
test_result = np.delete(jornal_test_arr, [1,2,3], axis=1)

In [ ]:
# Значення 'None' - не є числом, то заміним його на 0
test_result[test_result == None] = 0

In [ ]:
# залишити тільки статистично важливі показчики
test_result = np.delete(test_result, 0, axis=1)

In [ ]:
# будуємо словник з вихідними розрахунками
result_dict = {
    "кількість спостережень"   : len(test_result),
    "кількість пустих значень" : len(test_result) - np.count_nonzero(test_result),
    "середній бал"             : round(np.mean(test_result),1),
    "максимальний бал"         : np.max(test_result),
    "мінімальний бал"          : np.min(test_result[np.nonzero(test_result)]),
    "стандартне відхилення"    : round(np.std(test_result),1),
    "розмах вариації"          : np.max(test_result) - np.min(test_result[np.nonzero(test_result)]) 
    }

In [ ]:
result_dict

In [ ]:
# вивести шапку
print(
"""
=======================================
№  : ПОКАЗЧИК               : ЗНАЧЕННЯ     
=======================================
"""      
      )

# вивести результати аналізу датасета
i = 1
for key, value in result_dict.items():
    print (f'{i:<3} {key:<25}  {value}')
    i += 1

### ІНДИВІДУАЛЬНЕ ЗАВДАННЯ

Користуючись результатами, що отримані в [Лабораторній роботі № 3](https://shkliarskiy.moodlecloud.com/mod/page/view.php?id=1193) виконати процедури видобутку, збагачення та попереднього аналізу даних.

__Постановка__: В 3-й лабораторній роботі отримано показчик, що характеризує окрему властивість квартири (ціна, метраж та ін.). 
Необхідно:
1. Відобразити цей показчик на адресу квартири, яка знаходиться в файлі `street_name.csv`, тобто побудувати новий файл `street_flat_propery.csv` якій містить 2 колонки: _<назва вулиці> <ваш показчик>_
2. На основі цьго файлу зробити відповідну таблицю в _SQLite_ та збагатити цю дані в цій таблиці назвою району де розташована квартира (дані по районах додаються) 
3. Провести попередній аналіз отриманих даних - вивести первинні статистики показчика по районах (_середне_ , _мінімальне_ , _максимальне_ і т.д. - див. приклад вище)

In [1]:
# Злиття файлу з показчиком та файлу з назвами вулиць в новий файл

YOUR_FILE = 'prt_prices.txt'
with open(YOUR_FILE, encoding='utf-8') as f1, \
     open('street_name.csv', encoding='utf-8', newline='') as f2, \
     open('street_flat_propery.csv', 'w', encoding='utf-8') as f3:
    
    for x, y in zip(f1, f2):
        line = f'{x[:-1].strip()},{y[:-1].strip()}\n'
        print(line)
        f3.write(line)

115000.0,street

75000.0,Омеляновича-Павленка

140000.0,

140000.0,Ованеса Туманяна

160000.0,Петропавлівська

870000.0,Дніпровська

372096.575,Саксаганського

220000.0,Івана Мазепи

133000.0,Дніпровська

69000.0,Ділова

169999.0,Новодарницька

212000.0,Ревуцького

83500.0,Липківського

75000.0,Бендукідзе

65000.0,Введенська

82000.0,Виноградний

207000.0,Кайсарова

51990.0,Ахматової

57999.0,Радченка

57000.0,Дніпровська

95000.0,Ділова

50000.0,Бориспільська

95000.0,Болсуновська

46860.0,Українки

38500.0,Антонова

110000.0,Омеляновича-Павленка

269000.0,

49575.0,Ованеса Туманяна

119000.0,Петропавлівська

210000.0,Дніпровська

310000.0,Саксаганського

93000.0,Івана Мазепи

98000.0,Дніпровська

78000.0,Ділова

68000.0,Новодарницька

77000.0,Ревуцького

176000.0,Липківського

139300.0,Бендукідзе

129000.0,Введенська

136000.0,Виноградний

71000.0,Кайсарова

108000.0,Ахматової

175000.0,Радченка

45006.0,Дніпровська

78000.0,Ділова

65000.0,Бориспільська

84900.0,Болсуновська

68899.

#### імпортувати в БД отриманий файл 'street_flat_propery.csv' через CLI
- __sqlite3 districts.db__ - запустити SQLite та відкрити БД
- __.mode csv__ - перевести БД для роботи з csv-файлами
- __.import street_flat_propery.csv street_flat_propery__ - створює таблицю _street_flat_propery_ та завантажує в неї вміст файлу _street_flat_propery.csv_
- __SELECT * FROM street_flat_propery LIMIT 10;__ - вивести 10 записів для перевірки
- __.quit__ - завершити роботу з CLI SQLite



In [ ]:
# Підключитись до БД 'districts.db' в якій є прив'язка вулиць до районів 
conn = sqlite3.connect("districts.db")

# створити відповідний курсор
cur  = conn.cursor()

In [10]:
# для перевірки виведіть 10 записів з таблиці 'street_district'
result = cur.execute('SELECT * FROM street_district LIMIT 10').fetchall()
print(result)

[('9-го Травня', 'вулиця', 'Святошинський'), ('Абрикосова', 'вулиця', 'Оболонський'), ('Абрикосова', 'вулиця', 'Дарницький'), ('Абрикосова', 'вулиця', 'Подільський'), ('Августина Волошина', 'вулиця', 'Солом’янський'), ('Авдєєнка Генерала', 'вулиця', 'Святошинський'), ('Автозаводська', 'вулиця', 'Оболонський'), ('Автозаводський ', 'провулок', 'Оболонський'), ('Автопаркова', 'вулиця', 'Дарницький'), ('Авторемонтна', 'вулиця', "Солом'янський")]


In [11]:
# побудувати запит на створення таблиці 'result' з 3-ма полями:
#     district - текстове
#     street - текстове
#     property - числове
stmt = 'CREATE TABLE IF NOT EXISTS  result (\
    district TEXT,\
    street   TEXT,\
    property INT\
);'

In [22]:
# виконати запит
cur.execute(stmt)

In [15]:
# перевірити схему таблиці
cur.execute ("SELECT name FROM PRAGMA_TABLE_INFO('result');").fetchall()

[('district',), ('street',), ('property',)]

In [ ]:
# створити запит на злиття таблиці 'street_flat_propery' та 'street_district' 
# по полю 'street'
stmt_join = 'SELECT street_district.district, street_district.street, street_flat_propery.price FROM  \
            street_flat_propery LEFT JOIN street_district ON street_district.street = street_flat_propery.street'



In [ ]:
# перевірити запит
cur.execute(stmt_join).fetchall()

In [30]:
# створити запит для наповнення таблиці `result` результатом запиту на об'єднання
stmt_insert = 'INSERT INTO result ' + stmt_join

In [33]:
# виконати запит на заповнення
cur.execute(stmt_insert)

In [64]:
# сберегти вміст створеної таблиці у вигляді списка
result_list = cur.execute("SELECT district, property FROM result").fetchall()

_ = [print(x) for x in result_list]

(None, 115000)
(None, 75000)
(None, 140000)
(None, 140000)
('Оболонський', 160000)
('Подільський', 160000)
('Оболонський', 870000)
('Голосіївський, Печерський, Шевченківський', 372096.575)
(None, 220000)
('Оболонський', 133000)
('Голосіївський, Печерський', 69000)
(None, 169999)
('Дарницький', 212000)
(None, 83500)
(None, 75000)
('Подільський', 65000)
('Печерський', 82000)
("Голосіївський,Солом'янський", 207000)
(None, 51990)
(None, 57999)
('Оболонський', 57000)
('Голосіївський, Печерський', 95000)
('Дарницький', 50000)
('Печерський', 95000)
(None, 46860)
(None, 38500)
(None, 110000)
(None, 269000)
(None, 49575)
('Оболонський', 119000)
('Подільський', 119000)
('Оболонський', 210000)
('Голосіївський, Печерський, Шевченківський', 310000)
(None, 93000)
('Оболонський', 98000)
('Голосіївський, Печерський', 78000)
(None, 68000)
('Дарницький', 77000)
(None, 176000)
(None, 139300)
('Подільський', 129000)
('Печерський', 136000)
("Голосіївський,Солом'янський", 71000)
(None, 108000)
(None, 175000

#### для аналізу скористатися бібліотекою NumPy

In [65]:
import numpy as np

In [90]:
# створити np-масив з списку `result_list`
result_arr = np.array(result_list)
result_arr[result_arr == None] = 0
test_result = np.delete(result_arr, 0, axis=1)

In [89]:
# побудувати словник для результатів аналізу
result_dict = {
    "кількість спостережень"   : len(test_result),
    "середнє значення"         : round(np.mean(test_result),1),
    "максимальне значення"     : np.max(test_result),
    "мінімальне значення"      : np.min(test_result[np.nonzero(test_result)]),
    "стандартне відхилення"    : round(np.std(test_result),1),
    "розмах вариації"          : np.max(test_result) - np.min(test_result[np.nonzero(test_result)])
    }


In [92]:
# вивести шапку
print(
"""
================================================
№  :   РАЙОН        : ПОКАЗЧИК        : ЗНАЧЕННЯ     
================================================
"""      
      )

# вивести результати аналізу датасета
i = 1
for key, value in result_dict.items():
    print (f'{i:<3} {key:<25}  {value}')
    i += 1


№  :   РАЙОН        : ПОКАЗЧИК        : ЗНАЧЕННЯ     

1   кількість спостережень     812
2   середнє значення           163862.3
3   максимальне значення       4500000
4   мінімальне значення        19575
5   стандартне відхилення      258999.0
6   розмах вариації            4480425
